In [ ]:
# NOTES: 
    # Imputing "other" in water, toilet, floor w/ most common cluster value (as per DHS guidance)
    # Exluding motorboat, watch, animal cart from possible utensils due to high number of NAN
    # 82 hh report 10+ rooms; could be multi-generational hh, but # of hh members varies
    # Pakistan GPS data is incomplete

In [ ]:
import pandas as pd
import numpy as np
import timeit

from dhs_transform import get_counts, cat_recode, range_recode, multi_recode, recode, spot_check, check_cat_recodes

In [ ]:
# Read from zipped csv and check runtime
start = timeit.default_timer()
combined_dhs = pd.read_csv("s3://w210-poverty-mapper/dhs_data/transformed_data/combined_dhs.csv.zip", low_memory=False)
stop = timeit.default_timer()
print('Time: ', round((stop - start)/60, 2))

In [ ]:
# Define relevant column dictionary
relevant_columns = { "dhscc": "country", 
                    "hv001": "cluster", 
                    "LATNUM": "lat", 
                    "LONGNUM": "lon",
                    "hv025": "urban",
                    "hv009": "members",
                    "hv201": "water", 
                    "hv205": "toilet", 
                    "hv213": "floor", 
                    "hv216": "rooms", 
                    "hv206": "electricity", 
                    "hv208": "tv", 
                    "hv209": "fridge", 
                    "hv221": "landline", 
                    "hv243a": "mobile",
                    "hv212": "car", 
                    "hv210": "bike", 
                    "hv243e": "computer",
                    "hv211": "motorcycle_scooter", 
                    "hv243d": "motorboat", 
                    "hv243b": "watch",
                    "hv207": "radio", 
                    "hv243c": "animal_cart"}

# Keep relevant columns
combined_dhs_subset = combined_dhs[relevant_columns.keys()]
#combined_dhs_subset.head()

# Rename relevant columns
combined_dhs_subset = combined_dhs_subset.rename(
    columns=relevant_columns)
#combined_dhs_subset.head()

In [ ]:
# Describe relevant columns
combined_dhs_subset.describe()

In [ ]:
# Check value counts
count_cols = [col for col in combined_dhs_subset.columns if col not in ["cluster", "lat", "lon"]]
get_counts(count_cols)

In [ ]:
# Check cluster count per country
combined_dhs_subset.groupby("country")["cluster"].nunique()

In [ ]:
# Check "other" counts
other_cols = ["water", "toilet", "floor"]

print("Other value counts:")
for col in other_cols: 
    print("{}: {}".format(col, combined_dhs_subset[combined_dhs_subset[col] == 96].shape[0]))

In [ ]:
# Impute "other" w/most common cluster value
for col in other_cols:
    mode = combined_dhs_subset.groupby(["country", "cluster"])[col].transform(lambda x: x.mode()[0])
    combined_dhs_subset[col + "_impute"] = np.where(combined_dhs_subset[col] == 96, mode, combined_dhs_subset[col])

In [ ]:
# Count number of houses reporting 10+ rooms
combined_dhs_subset[combined_dhs_subset["rooms"] >= 10].shape[0]

In [ ]:
# Check household members for houses reporting 10+ rooms
combined_dhs_subset[combined_dhs_subset["rooms"] >= 10].groupby("rooms").agg({"members": ["mean", "min", "max"]})

In [ ]:
# Define recode dictionaries
# https://dhsprogram.com/pubs/pdf/DHSG4/Recode7_Map_31Aug2018_DHSG4.pdf
# See Pakistan and Philippines .map files for additional categories

# WATER SUPPLY
# high quality is private piped water or bottled water;
# middle quality is public tap or standpipe, tubewell or borehole, protected well or spring, or tanker truck;
# low quality is unprotected well or spring, cart with tank/drum, or surface water.
water_types = {"dict_type": "cat", 
               "var": "water_impute",
               "types":{"water_low": [32, 40, 42, 43, 51, 62, 96], 
                             "water_med": [14, 20, 21, 30, 31, 41, 61, 63], 
                             "water_high": [10, 11, 12, 13, 71]}}

# TOILET FACILITY
# high quality is private flush toilet (or flush toilet if private unknown);
# middle quality is public toilet, ventilated/improved pit latrine, pit latrine with slab, or composting toilet;
# low quality is pit latrine without slab, open pit, bucket, hanging toilet or no toilet.
toilet_types = {"dict_type": "cat", 
                "var": "toilet_impute", 
                "types": {"toilet_low": [20, 23, 30, 31, 42, 43, 96], 
                              "toilet_med": [21, 22, 41, 71], 
                              "toilet_high": [10, 11, 12, 13, 14, 15]}}

# FLOOR QUALITY
# high quality is finished floor with parquet, carpet, tiles, linoleum, ceramic etc.;
# middle quality is cement, concrete, wood, bamboo etc.;
# low quality is none, earth, dung etc.
floor_types = {"dict_type": "cat", 
               "var": "floor_impute",
               "types": {"floor_low": [11, 12, 96], 
                              "floor_med": [21, 22], 
                              "floor_high": [31, 32, 33, 34, 35, 36, 37, 38, 39]}}

# ROOMS USED FOR SLEEPING
# Zero or one; two; three or more
rooms = {"dict_type": "range", 
         "var": "rooms", 
         "types": {"rooms_zero_one": [0, 1], "rooms_two": [2, 2], "rooms_three_plus": [3, np.inf]}}

# PHONE
phone = {"dict_type": "multi_var", 
         "var": ["landline", "mobile"], 
         "types": {"phone": [1]}}
# Landline NAN (n=4); Mobile NAN (n=5)

# EXPENSIVE UTENSIL
# possession of expensive utensil (over 250 US Dollar)
expensive_utensil = {"dict_type": "multi_var", 
         "var": ["car", "computer", "motorcycle_scooter"], 
         "types": {"expensive_utensil": [1]}}

# CHEAP UTENSIL
# possession of cheap utensil (roughly under 50 US Dollar)
# include households with expensive utensil (see above), high quality floor or toilet facility, 
# TV, fridge, phone, bicycle, watch, radio, animal drawn cart
cheap_utensil = {"dict_type": "multi_var", 
         "var": ["expensive_utensil", "toilet_high", "floor_high", "tv", 
                 "fridge", "phone", "bike", "radio"], 
         "types": {"cheap_utensil": [1]}}


In [ ]:
# Create indicators derived from single variable
single_dict_list = [water_types, toilet_types, floor_types, rooms]
recode(single_dict_list)

In [ ]:
# Create indicators derived from multiple variables 
# (list order matters - these depend on each other and single variable recodes)
multi_dict_list = [phone, expensive_utensil, cheap_utensil]
recode(multi_dict_list)

In [ ]:
# Spot check recodes
#spot_check(water_types, 25)
#spot_check(toilet_types, 25)
#spot_check(floor_types, 25)
#spot_check(rooms, 25)
#spot_check(electricity, 1000)
#spot_check(tv, 25) 
#spot_check(fridge, 25)
#spot_check(car, 25) 
#spot_check(motorboat, 2000)
#spot_check(bike, 2000)
#spot_check(landline, 25)
#spot_check(mobile, 25)
#spot_check(computer, 25)
#spot_check(motorcycle_scooter, 25)
#spot_check(watch, 25) 
#spot_check(radio, 25)
#spot_check(animal_cart, 25)
#spot_check(phone, 25)
#spot_check(expensive_utensil, 25)
#spot_check(cheap_utensil, 25)

In [ ]:
# Check for double or skipped counting
cat_list = ["water", "toilet", "floor"]
check_cat_recodes(cat_list)

In [ ]:
# Set International Wealth Index (IWI) weights
# https://globaldatalab.org/iwi/using/
constant = 25.00447
weight_dict = {"tv": 8.612657, 
               "fridge": 8.429076, 
               "phone": 7.127699, 
               "car": 4.651382, 
               "bike": 1.84686, 
               "cheap_utensil": 4.118394,
               "expensive_utensil": 6.507283, 
               "floor_low": -7.558471, 
               "floor_med": 1.227531, 
               "floor_high": 6.107428, 
               "toilet_low": -7.439841,
               "toilet_med": -1.090393,
               "toilet_high": 8.140637,
               "rooms_zero_one": -3.699681, 
               "rooms_two": 0.38405, 
               "rooms_three_plus": 3.445009,
               "electricity": 8.056664,
               "water_low": -6.306477,
               "water_med": -2.302023,
               "water_high": 7.952443}

weight_columns = weight_dict.keys()
weight_values = [weight_dict[col] for col in weight_columns]

In [ ]:
# Check index input column counts
get_counts(weight_columns)

In [ ]:
# Count missing values by country
missing_count_cols = [item for item in weight_columns]
missing_count_cols.append("country")
combined_dhs_subset[missing_count_cols].groupby(
    ["country"]).count().rsub(
    combined_dhs_subset.groupby(["country"]).size(), axis=0)

In [ ]:
# Calculate household International Wealth Index (IWI)
combined_dhs_subset["wealth_index"] = constant + combined_dhs_subset[weight_columns].dot(weight_values)
#combined_dhs_subset.head()

In [ ]:
# Keep final columns
final_columns = ["country", "cluster", "lat", "lon", "urban", "wealth_index"] 
combined_dhs_subset_final = combined_dhs_subset[final_columns]
#combined_dhs_subset.head()

In [ ]:
# Count missing values by country
combined_dhs_subset_final.groupby(["country"]).count().rsub(combined_dhs_subset_final.groupby(["country"]).size(), axis=0)

In [ ]:
# Check missing count by country and cluster  
combined_dhs_subset_final[combined_dhs_subset_final["wealth_index"].isna()].groupby(["country", "cluster"])["cluster"].count()

In [ ]:
# Group by median cluster IWI
dhs_wealth_index = combined_dhs_subset_final.groupby(["country", "cluster"]).median().reset_index()

In [ ]:
# Check final dataset
dhs_wealth_index.head()

In [ ]:
# Check cluster counts
dhs_wealth_index.groupby("country")["cluster"].nunique()

In [ ]:
# Count missing values by country
dhs_wealth_index.groupby(["country"]).count().rsub(dhs_wealth_index.groupby(["country"]).size(), axis=0)

In [ ]:
# Write wealth index data as csv
dhs_wealth_index.to_csv("s3://w210-poverty-mapper/dhs_data/transformed_data/dhs_wealth_index.csv", index=False)